# Installation of Detectron **2**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!wget 'https://github.com/facebookresearch/detectron2/archive/refs/tags/v0.4.zip'

In [ ]:
!unzip "/content/v0.4.zip"

In [ ]:
%cd detectron2-0.4
!pip install -e .

# Setting Up Model

In [10]:
from detectron2.data.datasets.coco import DatasetCatalog
import torch
import detectron2
import torchvision
import cv2

from detectron2.structures import BoxMode
from detectron2.data import MetadataCatalog
from detectron2.data.datasets import register_coco_instances

detectron2.data.MetadataCatalog
path_data = '/content/drive/MyDrive/DetectronFolder/data/'

classes = ['scratch', 'cloudy', 'spot', 'radial']

for i in ["train" , "test"]:
  dataset_name = "data" + i
  if dataset_name in DatasetCatalog:
    DatasetCatalog.remove(dataset_name)
  if dataset_name in MetadataCatalog:
    MetadataCatalog.remove(dataset_name)

  register_coco_instances(
      "data" + i,
      {},
      path_data + i + f"/{i}.json",
      path_data + i,
  )
  metaDataTest = MetadataCatalog.get("data" + i).set(thing_classes=classes)

In [11]:
from detectron2 import model_zoo
from detectron2.engine import DefaultTrainer, DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import ColorMode, Visualizer
import os

cfg = get_cfg()

cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml"))

cfg.DATASETS.TRAIN = ("datatrain",)    # Set Training data

cfg.DATASETS.TEST = ()                    # Set Testing data

cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml")
cfg.SOLVER.IMS_PER_BATCH = 1             # 1 images per batch
cfg.SOLVER.BASE_LR = 0.00025             # Learning rate
cfg.SOLVER.MAX_ITER = 5000               # Iteration
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 4      # Number of classes
cfg.DATALOADER.NUM_WORKERS = 2           # Loading and preprocessing input

In [12]:
import os
os.makedirs(cfg.OUTPUT_DIR , exist_ok=True)
trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=False)      # Resume Training

[10/30 06:03:19 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

# Train Model

In [ ]:
trainer.train()

# Test Model

In [14]:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5
cfg.DATASETS.TEST = ("datatest", )
predictor = DefaultPredictor(cfg)

In [15]:
import json
import random
import cv2
import matplotlib.pyplot as plt
from detectron2.utils.visualizer import Visualizer, ColorMode

# Read the data from the JSON file
with open('/content/drive/MyDrive/DetectronFolder/data/test/test.json', 'r') as f:
    data = json.load(f)

# Extracting the images
images = data['images']

sample = random.sample(images , 50)

for img_data in sample:
    img_path = '/content/drive/MyDrive/DetectronFolder/data/test/' + img_data["file_name"]
    img = cv2.imread(img_path)
    outputs = predictor(img)

    v = Visualizer(img[:, :, ::-1],
                   metadata= metaDataTest,
                   scale=0.8,
                   instance_mode=ColorMode.IMAGE_BW)

    v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    os.makedirs('/content/drive/MyDrive/DetectronFolder/data/result/' , exist_ok=True)
    cv2.imwrite('/content/drive/MyDrive/DetectronFolder/data/result/' + img_data["file_name"], cv2.cvtColor(v.get_image(), cv2.COLOR_BGR2RGB))